### Oracle에서 bq로 데이터 적재(with Pandas)

In [13]:
from google.cloud import bigquery

GCP_PROJECT = "emart-datafabric"

ORA_OWNER = "C##DPC"
ORA_TABLE = "GTPSD_EP_PAY_RESULT_A"
PARTITION_COLUMN = "BIZ_DATE"

BQ_PARTITION_TYPE = bigquery.TimePartitioningType.DAY
BQ_PARTITION_FIELD = "dt"
BQ_CLUSTERING_FIELD = "pos_no"
BQ_DATASET = "temp_1d"
BQ_TABLE = "temp_gtpsd_ep_pay_result_a"

DT = '20220923'

In [14]:
from pydatafabric.vault_utils import get_secrets
from pydatafabric.oracle import set_oracle_secret, oracle_to_bq_overwrite_table

oracle_secret = get_secrets(mount_point="datafabric",path="oracle/datafabric/oracle-db")
set_oracle_secret(oracle_secret)

### bq 테이블에 overwrite

In [15]:
query = f"""
    select * 
    from {ORA_OWNER}.{ORA_TABLE}
    where {PARTITION_COLUMN} = '{DT}'
"""

oracle_to_bq_overwrite_table(owner=ORA_OWNER
                            , table=ORA_TABLE
                            , dataset=BQ_DATASET
                            , bq_table=BQ_TABLE
                            , bq_project=GCP_PROJECT
                            , partition_column=PARTITION_COLUMN
                            , partition_type=BQ_PARTITION_TYPE
                            , partition_field=BQ_PARTITION_FIELD
                            , query=query)